In [1]:
import numpy as np
import matplotlib.pyplot as plt

from data_handler import DataHandler
from log_regression import LogisticalRegression
from calculations import Calculations

TERMINATION_VALUE = 2^-32
ITERATIONS = 10000
LEARNING_RATE = 0.001
dh = DataHandler("spambase.data")
data = dh.parse_data_no_header()
data = dh.shuffle_data(data)
data_train, data_test = dh.split_data(data)

In [2]:
tX, tY, tPriors = dh.dynamic_split(data_train)
vX, vY, vPriors = dh.dynamic_split(data_test)
tX[0].shape, tX[1].shape, tY.shape, vX[0].shape, vX[1].shape, vY.shape, tPriors, vPriors

((1830, 57),
 (1237, 57),
 (3067, 1),
 (957, 57),
 (576, 57),
 (1533, 1),
 array([0.59667427, 0.40332573]),
 array([0.62426614, 0.37573386]))

In [3]:
def gnb(x_val, x_mean, x_var):
    equation_1 = 0
    equation_2 = 0
    if x_var != 0:
        equation_1 = 1/(np.sqrt(2 * np.pi * x_var))    

        e2denom = 2 * x_var
        e2num = np.square(x_val - x_mean)

        #   TODO: Fix Invalid Value / Check for Accuracy Changes
        #   expo = np.exp(-(np.divide(numerator, denom)))
        equation_2 = np.divide(e2denom, e2num)

    prob = equation_1 * equation_2
    return prob

probs = []
for i, group in enumerate(tX):
    prior = tPriors[i]
    for j in range(group.shape[1]):
        classX = group[:,j]
        gnb_ = gnb(classX, classX.mean(), classX.var())
    probs.append(np.multiply(gnb_, prior))

probs

[array([4.14635543e+00, 4.77752374e+00, 3.25843116e-03, ...,
        1.35918538e+01, 7.22588630e+00, 8.13370882e+00]),
 array([0.18980718, 1.50639507, 3.80939605, ..., 2.08796237, 1.81745961,
        0.69075983])]

In [7]:
preds = []
for i in range(len(probs)):
    prob = list(probs[i])
    for j in range(len(prob)):
        print(prob)
        max_index = prob.index(max(prob))
        preds.append(tY[max_index, 0])
    
preds

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0